In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
df = pd.read_pickle("gujsemicompiled.pickle") 
df.head()

,City,Population,Population_Density,Total Talukas,Latitude,Longitude,UrbanPopPercentage,GrowthRate,SexRatio,LiteracyRate,Total Schools,Teachers,SchoolsWithPlaygrounds,SchoolsWithComputers
0,AHMEDABAD,7045313,983,10,23.021624,72.579707,28.30,22.31,903.0,86.65,2912,28401,2671,2582
1,AMRELI,1513614,224,11,22.373810,73.044198,41.01,8.59,964.0,74.49,1041,8032,911,959
2,ANAND,2090276,446,8,22.563200,72.960280,22.76,12.57,921.0,85.79,1382,10633,1083,1029
3,PALANPUR,3116045,245,14,21.208740,72.776320,20.93,24.43,936.0,66.39,2730,19361,2031,1854
4,BHARUCH,1550822,238,9,21.701400,72.962690,79.68,13.14,924.0,83.03,1221,8949,851,1008


In [4]:
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [5]:
import requests # library to handle requests
import random # library for random number generation

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize


In [44]:
CLIENT_ID = 'U5WZX01K4NQUZKHZRNNB0KRPYXKK0WVTIR0HT5KQYY5TQEEE' # your Foursquare ID
CLIENT_SECRET = 'J1JMWSIMMA3F4DMMVX2K1EJAYBVNKX0DUDP2SE4LJNZF5XYH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500

In [40]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    


In [48]:
radius = 10000
lat = 23.021624
long = 72.579707
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, radius, LIMIT)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
    
dataframe = json_normalize(items) # flatten JSON
filtered_columns = ['venue.name', 'venue.categories'] + ['venue.location.city', 'venue.location.state']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
dataframe_filtered.head(10)
    

,name,categories,city,state
0,Manek Chowk Khau Gali,Snack Place,Ahmedabad,Gujarāt
1,Manek Chowk,Fast Food Restaurant,Ahmedabad,Gujarāt
2,Swati Snacks,Snack Place,Ahmedabad,Gujarāt
3,Crossword,Bookstore,Ahmedabad,Gujarāt
4,TOMATO'S,Mexican Restaurant,Ahmedabad,Gujarāt
5,The Sports Club of Gujarat,Cricket Ground,Ahmedabad,Gujarāt
6,Lucky Tea,Tea Room,Ahmedabad,Gujarāt
7,Havmor,Indian Restaurant,Ahmedabad,Gujarāt
8,Havmor,Ice Cream Shop,Ahmedabad,Gujarāt
9,Dunkin' Donuts,Donut Shop,Ahmedabad,Gujarāt


In [60]:
dataframe_filtered = dataframe_filtered.drop(dataframe_filtered[dataframe_filtered.categories=='Café'].index)
dataframe_filtered = dataframe_filtered.drop(dataframe_filtered[dataframe_filtered.categories.str.contains("Restaurant")].index)
dataframe_filtered = dataframe_filtered.drop(dataframe_filtered[dataframe_filtered.categories.str.contains("Snack")].index)

In [61]:
dataframe_filtered['name'].count()

54

In [65]:
radius = 10000
mylist = []
for city, i in zip(df['City'], range(0, len(df.index))):
    lat = df.iloc[i]['Latitude']
    long = df.iloc[i]['Longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
    .format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    items = results['response']['groups'][0]['items']
    
    dataframe = json_normalize(items) # flatten JSON
    filtered_columns = ['venue.name', 'venue.categories'] + ['venue.location.city', 'venue.location.state']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
    #dataframe_filtered = dataframe_filtered.drop(dataframe_filtered[dataframe_filtered.categories=='Café'].index)
    #dataframe_filtered = dataframe_filtered.drop(dataframe_filtered[dataframe_filtered.categories.str.contains("Restaurant")].index)
    #dataframe_filtered = dataframe_filtered.drop(dataframe_filtered[dataframe_filtered.categories.str.contains("Snack")].index)
    venue_counts = dataframe_filtered['name'].count()
    mylist.append([city,venue_counts])


In [66]:
df_venuecounts = pd.DataFrame(mylist, columns=['City', 'Venue Counts'])
df_venuecounts

,City,Venue Counts
0,AHMEDABAD,100
1,AMRELI,4
2,ANAND,19
3,PALANPUR,92
4,BHARUCH,7
5,BHAVNAGAR,50
6,AHWA,1
7,GANDHINAGAR,24
8,JAMNAGAR,14
9,JUNAGADH,5


In [67]:
df_venuecounts.to_pickle("gujcityvenuecounts.pickle")
df_venuecounts.to_csv("gujcityvenuecounts.csv")